# Session #2
## ML-based SA by using Naive Bayes

두 번째 세션에서는 Naive Bayes Calssifier 를 이용한 영어 텍스트 감성분석 프로그램을 작성해 봅니다. 
실습 수업은 프로그램의 주요 흐름을 설명하면서, TODO 처리된 핵심적인 부분의 코드를 직접 작성해보는 순서로 진행될 것입니다.  

In [1]:
from __future__ import print_function # python 2 and 3 compatibility
import io
import nltk
from nltk import word_tokenize
from os import listdir
import math
import six

#-*- coding: utf-8 -*-

In [2]:
# download nltk package for word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### **구글 드라이브를 이용하여 데이터 import 및 압축풀기**
a) 내 구글 드라이브 mount 하기


In [3]:
""" 
내 구글 드라이브와 colab 연결 ==> 
아래 셀 실행 후 출력되는 링크 클릭 ==> 
authorization code 복사 후 아래 출력된 박스에 붙여넣고 enter 키 입력
"""
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
"""
내 구글 드라이브와 잘 연동 되었는지 확인해보기
실습 파일들을 내 구글 드라이브 (My Drive) 내에 다른 폴더를 만들었다면 ==> 
os.listdir('/content/drive/My Drive/내가_만든_폴더_이름')
"""
import os
os.listdir('/content/drive/My Drive/Colab Notebooks/20.10.30')

['6humanCodedDataSets.zip',
 'SentiStrength_Data.zip',
 'data.zip',
 'session1_dictionary_SA.ipynb',
 '2020_2학기_assignment_session1_dictionary_SA.ipynb',
 'session2_NaiveBayes_SA.ipynb',
 '2020_2학기_assignment_session2_NaiveBayes_SA.ipynb']

In [5]:
"""
구글 드라이브에 올려놓은 실습 데이터 zip 압축 풀기 ==> 
zipfile 의 extractall 함수 이용 
압축을 풀 경로 (directory_to_extract_to)는 반드시 '/tmp/' 아래 지정할 것 ex) '/tmp/data/'
"""
import zipfile
path_to_zip_file = '/content/drive/My Drive/Colab Notebooks/20.10.30/data.zip'
directory_to_extract_to = '/tmp/data'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
  zip_ref.extractall(directory_to_extract_to)

## variable 선언 부
- 확률 계산 값을 저장할 변수 및 file path 를 설정합니다.

In [6]:
# Variables

voca_dic = {}             # token dictionary
log_prior_pos = 1         # prior probability for positive class
log_prior_neg = 1         # prior probability for negative class

log_likelihood_pos = {}   
log_likelihood_neg = {}

## step 1. Compute the prior probability
- 각 class 의 (positive, negative) prior 확률을 구해봅니다.
- training set 상에서 positive / negative class 에 속하는 data 의 개수를 count 하는 방법으로 구할 수 있습니다.
- 본 실험에서는 file 의 개수를 count 하는 것으로 구해봅니다.

In [7]:
'''
count # of files in class path
Input: data path
Output: # of data
'''
def count_file(dir_path):
    
    '''
    -- TODO -- 
    list files = ... 
    '''

    list_files = [f for f in listdir(dir_path)]
    
    return len(list_files)

사전 데이터와 테스트에 사용할 텍스트 파일이 있는 폴더를 지정합니다.  

마운트된 구글 드라이브 사용하여 압축 해제한 폴더 지정 (/tmp/data/)

In [8]:
data_root_path = '/tmp/data'

#target_data = data_root_path + '/sample'
target_data = data_root_path + '/train'

log_prior_pos = math.log(count_file(target_data + '/pos'))
log_prior_neg = math.log(count_file(target_data + '/neg'))

print('* log prior of the positive class: ' + str(log_prior_pos))
print('* log_prior of the negative class: ' + str(log_prior_neg ))

* log prior of the positive class: 9.433483923290392
* log_prior of the negative class: 9.433483923290392


## step 2. Compute the Likelihood
- 주어진 문장과 각 class 간 likelihood 값을 계산해 봅니다

### step 2-1. Create Vocabulary Dictionary
- 현재 data set 에서 사용되는 모든 token 을 파악하기 위해서 전체 data set 에 들어 있는 token 을 포함하는 dictionary 를 생성합니다.
- 특정 폴더 안에 있는 포든 파일을 읽어서 해당 파일에 들어 있는 문장을 tokenize 후 얻어진 token 을 dictioinary 에 추가 합니다.

In [9]:
'''
build token dictionary
Input: data path, 최종 dictionary
Return: None (dictionary 는 내부에서 update 됨)
'''
def build_dic(dir_path, dic):
    
    list_files = [f for f in listdir(dir_path)]

    for file in list_files:
      
        try:
            f = open(dir_path + file, 'r')
            
            '''
            -- TODO -- 
            dir_path 내에 있는 파일들을 loop
      
            하나의 파일에 있는 내용을 읽은 후 tokenize
            ex) tokens = word_tokenize(line.strip().lower())

            tokens 에 담겨 있는 token 들을 dictionary 에 추가   
            '''

            line = f.readline()
            tokens = word_tokenize(line.strip().lower())

            for token in tokens:
              if token not in dic:
                dic[token] = 1

            f.close()        
        
        except:
            pass

positive, negative class 안에 들어 있는 모든 data 에서 token 을 추출하여 dictionary 를 완성 합니다.

In [10]:
build_dic(target_data + '/pos/', voca_dic)
build_dic(target_data + '/neg/', voca_dic)
print("* total voca size: " + str(len(voca_dic)))

* total voca size: 114526


### Step 2-2. 각 Class 별 token 의 확률 table 생성
- 특정 class 내 data 를 모두 tokenize 한 후 각 token 의 수를 count 하여 해당 token 이 해당 class 에서 나타날 확률을 계산합니다.
- 확률 값 계산시 제외 되는 token 이 없게 하기 위해 전체 dictionary 를 기본으로 가진 후 추가 계산을 합니다.

In [11]:
'''
class 별 확률 table 생성
Input: data path, 전체 dictionary
Return: 해당 class 의 확률 테이블
'''
def create_class_dic(dir_path, base_dic):
    
    # copy base_dic and create likelihood_table
    likelihood_table = {}
    likelihood_table = dict( (nkey, 1) for nkey in [key for key in base_dic.keys()])
        
    list_files = [f for f in listdir(dir_path)]

    for file in list_files:
      
        try:
            f = open(dir_path + file, 'r')
            line = f.readline()  
            tokens = word_tokenize(line.strip().lower())

            for token in tokens:
                
                '''
                -- TODO -- 
                likelihood table 을 update
                '''
                likelihood_table[token] = likelihood_table[token] + 1

            f.close()     
            
        except:
            pass
            
    return likelihood_table

In [12]:
pos_table = create_class_dic(target_data + '/pos/', voca_dic)  
neg_table = create_class_dic(target_data + '/neg/', voca_dic)  

token = 'good'
print('token = ' + token)
print('# of tokens in positive class: \t' + str(pos_table[token]))
print('# of tokens in negative class: \t' + str(neg_table[token]))

token = good
# of tokens in positive class: 	7445
# of tokens in negative class: 	7196


### Step 2-3. 확률 table 을 log 확률 값으로 변환
- log 함수는 monotonically increasing 함수이므로 log 를 취한 값으로 확률을 계산해도 동일한 비교가 가능합니다.
- 확률값 계산시 * (곱셈) 이 아닌 + (덧셈) 으로 계산 가능하기 때문에 연산이 수월 합니다.
- 곱셈으로 확률을 계산시 확률값이 매우 작아 질 경우 발생하는 수치에러를 방지할 수 있습니다.

In [13]:
'''
log likelihood table 연산
Input: 특정 class 의 확률 테이블
Return: # of data
'''
def compute_log_likelihood_table(class_table):
    
    new_table = {}
    word_sum = sum(class_table.values())
    new_table = dict( (key, math.log((float)(value)/(float)(word_sum)) ) for (key, value) in six.iteritems(class_table)) 
    
    return new_table

In [14]:
log_likelihood_pos = compute_log_likelihood_table(pos_table)
log_likelihood_neg = compute_log_likelihood_table(neg_table)

token = 'good'
print('token = ' + token)
print('# of token in class: \t\t\t' + str(pos_table[token]))
print('probability of the token in class: \t' + str(pos_table[token] / float(sum(pos_table.values()))))
print('log probability: \t\t\t' + str(log_likelihood_pos[token]))

token = good
# of token in class: 			7445
probability of the token in class: 	0.0020286954820447505
log probability: 			-6.20036231217936


## Step 3. 하나의 문서를 분류해 보세요

- 클래스별 log_table 을 이용해서 classifier 를 구현해 보세요
- positive class 에 속할 확률 vs negative class 에 속할 확률

In [15]:
'''
특정 document 에 들어 있는 문장을 분류 (positive/negative)
Input: document, positive class prior 확률, negative class prior 확률, positive class likelihood 테이블, negative class likelihood 테이블
Return: 클래스
'''
def classify_doc(document, log_prior_pos, log_prior_neg, log_likelihood_pos, log_likelihood_neg):
    
    pos_prob = 0
    neg_prob = 0
    
    tokens = word_tokenize(document.strip().lower())

    for token in tokens:

        '''
        -- TODO -- 
        pos_prob 에 주어진 토큰의 해당 클래스에 따른 확률 값을 누적
        neg_prob 에 주어진 토큰의 해당 클래스에 따른 확률 값을 누적
        '''
        if token in log_likelihood_pos:
          pos_prob += log_likelihood_pos[token]
        if token in log_likelihood_neg:
          neg_prob += log_likelihood_neg[token]

    pos_prob = pos_prob + log_prior_pos       
    neg_prob = neg_prob + log_prior_neg

    if pos_prob > neg_prob:
        return 'positive' 
    else:
        return 'negative'

In [16]:
document = 'this is  my favorite. so exciting'
# document = 'i hate the move.. jsut waste of the time'

ret = classify_doc(document, log_prior_pos, log_prior_neg, log_likelihood_pos, log_likelihood_neg)

print('  input document  : \t' + document)
print('  predicted class   : \t' + ret)

  input document  : 	this is  my favorite. so exciting
  predicted class   : 	positive


## Step 4. 주어진 path 에 있는 모든 문서를 분류
- 특정 폴더 안에 있는 모든 문서를 분류하고, 정확도를 측정해 본니다.
- 주어진 폴더 안에는 같은 class 의 data 가 분류되어 들어 있습니다. 
- Train 시에 사용되지 않은 data 를 가지고 테스트를 해야 합니다.

In [17]:
'''
특정 폴더 안에 있는 모든 data 분류
Input: data path
Return: None
'''
def evaluate_all(dir_path):

    list_files = [f for f in listdir(dir_path)]

    pos_cnt = 0
    neg_cnt = 0
    process_doc = 0

    for file in list_files:
        '''
        -- TODO -- 
        파일을 읽은 후 위에서 작성한 classify_doc() 함수를 이용하여 class 를 분류
        '''
        try:
            f = open(dir_path + file, 'r')
            line = f.readline()
            result = classify_doc(line, log_prior_pos, log_prior_neg, log_likelihood_pos, log_likelihood_neg)
            if(result == 'positive'):
              pos_cnt += 1
            else:
              neg_cnt += 1
            process_doc += 1
            # f.close()
        except:
            pass
    print(f"Classify Summary for {dir_path} : ")
    print(f"Total classified documents : {process_doc} documents")
    print(f"positive counted documents: {pos_cnt} documents")
    print(f"negative counted documents: {neg_cnt} documents")

In [18]:
evaluate_all(data_root_path + '/test/neg/')
evaluate_all(data_root_path + '/test/pos/')    

Classify Summary for /tmp/data/test/neg/ : 
Total classified documents : 12500 documents
positive counted documents: 1559 documents
negative counted documents: 10941 documents
Classify Summary for /tmp/data/test/pos/ : 
Total classified documents : 12500 documents
positive counted documents: 9297 documents
negative counted documents: 3203 documents
